In [1]:
import json
import sys
from pandarallel import pandarallel

pandarallel.initialize(progress_bar=True, verbose=0, nb_workers=10)


sys.path.append('../')

from utils import (
    load_json,jsonl_dir_to_df,
    load_simple_json,generate_evidence_to_wiki_pages_mapping
)


In [30]:
no_info_page = load_json("../pert_large/page/0522_base_clu_folds/no_info_ens5.jsonl")
len(no_info_page)

3268

In [31]:
no_info_sent = load_json("../pert_large/sent/0522_base_clu_folds_PN13/no_info_ens5.jsonl")
len(no_info_data)

2686

In [63]:
all_no_info = load_json("../preprocess/pre_all_0522.jsonl")
all_no_info = [i for i in all_no_info if i["label"]=="NOT ENOUGH INFO"]
len(all_no_info)

3275

In [52]:
test_emb = load_json("../preprocess/pre_train_wikisearch_base.jsonl")
emb_map = {i["id"]:idx for idx,i in enumerate(test_emb)}

{'id': 8111,
 'result': ['言論自由',
  '中華民國言論自由',
  '信息自由',
  '網路用戶',
  '網路論戰',
  'IBM系統網路架構',
  '身分標識方式',
  '身分證',
  '中華民國國民身分證',
  '訴諸言論自由'],
 'correct': [None]}

In [6]:
2686+325

3011

In [36]:
no_info_page[0]

{'id': 8075,
 'label': 'NOT ENOUGH INFO',
 'claim': 'F.I.R.的團員有主唱Faye飛（詹雯婷）、吉他手Real阿沁（黃漢青）、鍵盤手Ian（陳建寧），是亞洲樂壇不常見的一女二男三人組合樂團。',
 'evidence': [[7208, None, None, None]],
 'predicted_pages_1': ['F.I.R.飛兒樂團']}

In [45]:
len(no_info_sent)

3268

In [44]:
def join_page_sent(page_data,sent_data):
    page_map = {i["id"]:idx for idx,i in enumerate(page_data)}
    sent_map = {i["id"]:idx for idx,i in enumerate(sent_data)}
    for idx in page_map:
        if idx not in sent_map:
            p_buf = page_data[page_map[idx]].copy()
            p_buf["predicted_evidence"]=[]
            sent_data.append(p_buf)

In [42]:
join_page_sent(no_info_page,no_info_sent)

In [48]:
wiki_pages = jsonl_dir_to_df("../../baseline/data/wiki-pages")
mapping = generate_evidence_to_wiki_pages_mapping(wiki_pages)
del wiki_pages

Reading and concatenating jsonl files in ../../baseline/data/wiki-pages
Generate parse mapping


Transform to id to evidence_map mapping


In [67]:
def map_to_sent(mapping,data):
    for i in data:
        sent_buf=[]
        for pred_evid in i["predicted_evidence"]:
            evid_id,evid_idx=pred_evid[0],pred_evid[1]
            try:
                sent_buf.append(mapping[evid_id][str(evid_idx)])
            except:
                sent_buf.append([])
        i["predicted_sent"]=sent_buf

In [49]:
map_to_sent(mapping,no_info_sent)
no_info_sent[0]

{'id': 8075,
 'label': 'NOT ENOUGH INFO',
 'claim': 'F.I.R.的團員有主唱Faye飛（詹雯婷）、吉他手Real阿沁（黃漢青）、鍵盤手Ian（陳建寧），是亞洲樂壇不常見的一女二男三人組合樂團。',
 'evidence': [[7208, None, None, None]],
 'predicted_pages_1': ['F.I.R.飛兒樂團'],
 'predicted_evidence': [['F.I.R.飛兒樂團', 0],
  ['F.I.R.飛兒樂團', 1],
  ['F.I.R.飛兒樂團', 2]],
 'predicted_sent': ['F.I.R. ， 爲華語樂壇少見一女二男三人組合樂團 ， 由主唱Faye飛 （ 詹雯婷 ） 、 吉他手Real阿沁 （ 黃漢青 ） 、 鍵盤手Ian （ 陳建寧 ） 組成 。',
  '2018年Faye飛被其餘二人單方面宣佈她已經離團 ， 隨後Faye飛曾於公開場合表示看到新聞才知道被離團 ， 但祝福對方 。',
  '同年由Lydia （ 韓睿 ） 加入並接替其主唱位置 ， 但歌唱實力受到一定程度的質疑 。']}

In [55]:
for j in [i for i in no_info_sent if i["predicted_evidence"]==[]]:
    print(j["claim"],test_emb[emb_map[j["id"]]]["result"])

網路使用者由於少了真實身分，因此有更多的意見自由。 ['言論自由', '中華民國言論自由', '信息自由', '網路用戶', '網路論戰', 'IBM系統網路架構', '身分標識方式', '身分證', '中華民國國民身分證', '訴諸言論自由']
每個禮拜的第七個太陽日稱爲星期六。 ['太陽日', '真太陽', '平太陽', '星期', '國際固定歷', '伊斯蘭曆', '禮拜_(伊斯蘭教)', '德國電視', '禮拜站', '日']
德布西在兒童時展現了天賦，而他出生於一個簡樸的家庭。 ['兒童', '兒童性虐待', 'TVB兒童臺', '德布西森林', '德布西撞擊坑', '託雷德布西', '有線兒童臺', '聯合國兒童基金會', '天賦海灣', '張天賦']
網路使用者由於少了真實身分，因此有更多的言語自由。 ['霸凌', '網路用戶', '網路論戰', 'IBM系統網路架構', '身分標識方式', '身分證', '中華民國國民身分證', '美國公民自由聯盟', '仇恨言論', '網路白目']
米夏埃爾·科爾曼（MichaelKohlmann)是一位德裔職業網球運動員，他也是一位投資客，目前的他擁有五棟房子，他成爲最高世界排名27位的富豪。 ['米沙·茲韋列夫', '安德烈·阿加西', '約翰·麥肯羅', '張德培', '歐裔美國人', '凱文·安德森_(網球運動員)', '德裔美國人', '王鈺_(網球運動員)', '德裔賓夕法尼亞人', '必勝客']
徐太志離婚後又結仇。 ['徐太志', '徐太志和孩子們', '徐元太', '徐志焄']
墊子藝術大獎的獎項分成三個級別。 ['Wii', '獎勵', '年度歌曲獎_(金曲獎)', '倫敦聖三一學院', '最佳華語專輯獎_(金曲獎)', '最佳華語女歌手獎_(金曲獎)', '最佳新人獎_(金曲獎)', '中華職棒獎項', '金馬獎獎項列表', '世界羽聯第三等別賽事']
日本遊戲公司伊藤洋華堂在2005年改組成日本7-11附屬公司。 ['7&I控股', '任天堂', 'Sting_(公司)', '世嘉', '寶可夢公司', '日本成人遊戲', '日本Falcom', '伊藤洋華堂', '鈴木敏文', '2005年世界博覽會']
網路使用者由於少了真實天分，因此有更多的輿論自由。 ['網路用戶', '網路論戰',

In [56]:
with open("./no_info_data.jsonl","w",encoding="utf8",) as f:
    for i, d in enumerate(no_info_sent):
        f.write(json.dumps(d, ensure_ascii=False) + "\n")

enough

In [8]:
all_data = load_json("../preprocess/pre_all_0522.jsonl")
all_pos = [i for i in all_data if i["label"]!="NOT ENOUGH INFO"]
len(all_pos)

8074

In [9]:
all_pos[0]

{'id': 2663,
 'label': 'refutes',
 'claim': '天衛三軌道在天王星內部的磁層，以《仲夏夜之夢》作者緹坦妮雅命名。',
 'evidence': [[[4209, 4331, '天衛三', 2]]]}

In [57]:
def map_to_sent_evid(mapping,data):
    for i in data:
        total_evid = []
        evid_set = i["evidence"]
        for evids in evid_set:
            for evid in evids:
                total_evid.append(evid[2:])
        total_evid=unique(total_evid)
        i["predicted_evidence"]=total_evid
        sent_buf=[]
        for evid in total_evid:
            try:
                evid_id,evid_idx=evid[0],evid[1]
                sent_buf.append(mapping[evid_id][str(evid_idx)])
            except:
                sent_buf.append([])
        i["predicted_sent"]=sent_buf

In [58]:
map_to_sent_evid(mapping,all_pos)

In [61]:
len(all_pos+no_info_sent)

11342

In [62]:
with open("./all_data.jsonl","w",encoding="utf8",) as f:
    for i, d in enumerate(all_pos+no_info_sent):
        f.write(json.dumps(d, ensure_ascii=False) + "\n")

In [17]:
def unique(input_lst):
    lst_set = []
    for i in input_lst:
        if i not in lst_set:
            lst_set.append(i)
    return lst_set

In [27]:
len(all_pos)

8074

In [88]:
dd = load_json("public_private_test_ans_cluster_r3_nl.jsonl")

In [89]:
def map_to_sent_PH(mapping,data):
    for i in data:
        if i["predicted_label"]=="NOT ENOUGH INFO":
            i["predicted_sent"]=[]
            continue
        sent_buf=[]
        for pred_evid in i["predicted_evidence"]:
            evid_id,evid_idx=pred_evid[0],pred_evid[1]
            try:
                sent_buf.append(mapping[evid_id][str(evid_idx)])
            except:
                sent_buf.append([])
        i["predicted_sent"]=sent_buf

In [90]:
map_to_sent_PH(mapping,dd)

In [92]:
dd[:5]

[{'id': 5208,
  'predicted_label': 'NOT ENOUGH INFO',
  'predicted_evidence': [],
  'predicted_sent': []},
 {'id': 1019,
  'predicted_label': 'supports',
  'predicted_evidence': [['昆蟲', 23], ['昆蟲', 22]],
  'predicted_sent': ['蠶產絲 ， 蜜蜂產蜂蜜 ， 兩者都已被人類馴化 。',
   '許多昆蟲被認爲是對生態有益的捕食者 ， 少數昆蟲提供直接的經濟利益 。']},
 {'id': 8514,
  'predicted_label': 'refutes',
  'predicted_evidence': [['綠山城縣', 0]],
  'predicted_sent': ['綠山城縣  ， 是波蘭的縣份 ， 位於該國西部 ， 由盧布斯卡省負責管轄 ， 首府設於綠山城 ， 面積 1,571 平方公里 ， 2006年人口 89,543 ， 人口密度每平方公里57人 。']},
 {'id': 1874,
  'predicted_label': 'supports',
  'predicted_evidence': [['魂斷藍橋', 0]],
  'predicted_sent': ['《 魂斷藍橋 》 （ Waterloo Bridge ） 是美國黑白電影 ， 由米高梅電影公司於1940年出品 ； 導演是茂文 · 李洛埃 （ Mervyn LeRoy ） ， 女主角是主演 《 亂世佳人 》 的費雯 · 麗 （ Vivien Leigh ） 而男主角是羅伯特 · 泰勒 （ Robert Taylor ） 。']},
 {'id': 8352,
  'predicted_label': 'supports',
  'predicted_evidence': [['侯孝賢', 2], ['刺客聶隱娘', 0], ['侯孝賢', 0], ['刺客聶隱娘', 2]],
  'predicted_sent': ['2015年以 《 刺客聶隱娘 》 獲得第68屆坎城影展最佳導演獎及第52屆金馬獎最佳導演獎與金馬獎最佳劇情片獎 。',
   '《 刺客聶隱娘

In [94]:
with open("./public_private_test_ans_cluster_r3_nl_sented.jsonl","w",encoding="utf8",) as f:
    for i, d in enumerate(dd):
        f.write(json.dumps(d, ensure_ascii=False) + "\n")

In [95]:
len(dd)

9038

In [61]:
pub_test =load_json("../pert_large/sent/0522_base_clu_folds_PN13/pub_test_ens5.jsonl")
#pub_test =load_json("../preprocess/public_test.jsonl")

In [62]:
len(pub_test)

921

In [102]:
map_to_sent(mapping,pub_test)

In [103]:
pub_test[0]

{'id': 5208,
 'claim': '光學顯微鏡是以電磁學原理來將不可見或難見的微小物放大至肉眼可見的儀器。',
 'predicted_pages_1': ['顯微鏡', '光學顯微鏡'],
 'predicted_evidence': [['顯微鏡', 0],
  ['光學顯微鏡', 0],
  ['顯微鏡', 6],
  ['顯微鏡', 1],
  ['光學顯微鏡', 3]],
 'predicted_sent': ['顯微鏡泛指將微小不可見或難見物品之影像放大 ， 而能被肉眼或其他成像儀器觀察之工具 。',
  '光學顯微鏡 （ Optical microscope 、 Light microscope ） 是一種利用光學透鏡產生影像放大效應的顯微鏡 。',
  '最常見的 （ 和第一個被髮明的 ） 是光學顯微鏡 ， 其他主要的顯微鏡類型包括電子顯微鏡 、 掃描探針顯微鏡等 。',
  '日常用語中之顯微鏡多指光學顯微鏡 ， 放大倍率和清析度 （ 聚焦 ） 爲顯微鏡重要因素 。',
  '由物體入射的光被至少兩個光學系統 （ 物鏡和目鏡 ） 放大 。']}

In [24]:
with open("./pub_test_ens5_sented.jsonl","w",encoding="utf8",) as f:
    for i, d in enumerate(pub_test):
        f.write(json.dumps(d, ensure_ascii=False) + "\n")

NameError: name 'pub_test' is not defined

### deal with probability and map sent for PH


In [65]:
data =load_json("all_test_clu_page_sent_claim_nclu_ens5_probs.jsonl")

In [66]:
DATA_MAP = {i["id"]:idx for idx,i in enumerate(data)}
pub_test_map = {i["id"]:idx for idx,i in enumerate(pub_test)}

In [67]:
pub_test_extract=[]
for i in DATA_MAP:
    if i in pub_test_map:
        pub_test_extract.append(data[DATA_MAP[i]])

In [68]:
len(pub_test_extract)

921

In [123]:
def top_n(data,n):
    re_lst=[]
    prob_map = {i["id"]:abs(i["enough"]-0.5) for idx,i in enumerate(data)}
    data_map = {i["id"]:idx for idx,i in enumerate(data)}
    sort_set = sorted(prob_map.items(), key=lambda x:x[1])
    for i in sort_set[:n]:
        re_lst.append(data[data_map[i[0]]])
    return re_lst

In [135]:
top_100 = top_n(pub_test_extract,100)

In [136]:
map_to_sent(mapping,top_100)

In [137]:
top_100[1]

{'id': 1399,
 'claim': '艾根訴加拿大案是人民對同性戀權利舉辦辯論會。',
 'predicted_pages_1': ['艾根訴加拿大案'],
 'predicted_evidence': [['艾根訴加拿大案', 0],
  ['艾根訴加拿大案', 15],
  ['艾根訴加拿大案', 7],
  ['艾根訴加拿大案', 22],
  ['艾根訴加拿大案', 8]],
 'enough': 0.49007536461576817,
 'predict_label': 0.9805730938911438,
 'predicted_sent': ['艾根訴加拿大案 （ Egan v. Canada ） [ 1995 ] 2 S.C.R. 513是加拿大最高法院圍繞同性戀權利而作出的一個判決 。',
  '這使得這個判決成爲一個具有爭議性的 ， 要求給予同性戀權利保障的判決書 。',
  '艾根隨之起訴 。',
  '這個判決成爲一個先例 ： 《 加拿大權利與自由憲章 》 第十五部分禁止基於性傾向的歧視 。',
  '在地方法院和聯邦上訴法院輸掉了官司以後 ， 艾根和奈斯比特上訴至加拿大最高法院 。']}

In [138]:
with open("./lowest_pub_100_enough.jsonl","w",encoding="utf8",) as f:
    for i, d in enumerate(top_100):
        f.write(json.dumps(d, ensure_ascii=False) + "\n")

In [69]:
with open ("lowest_pub_100_enough_PH.jsonl","r") as f:
    data= f.readlines()
data[1]

'refute or not enough,辯論會{"id": 1399, "claim": "艾根訴加拿大案是人民對同性戀權利舉辦辯論會。", "predicted_pages_1": ["艾根訴加拿大案"], "predicted_evidence": [["艾根訴加拿大案", 0], ["艾根訴加拿大案", 15], ["艾根訴加拿大案", 7], ["艾根訴加拿大案", 22], ["艾根訴加拿大案", 8]], "enough": 0.49007536461576817, "predict_label": 0.9805730938911438, "predicted_sent": ["艾根訴加拿大案 （ Egan v. Canada ） [ 1995 ] 2 S.C.R. 513是加拿大最高法院圍繞同性戀權利而作出的一個判決 。", "這使得這個判決成爲一個具有爭議性的 ， 要求給予同性戀權利保障的判決書 。", "艾根隨之起訴 。", "這個判決成爲一個先例 ： 《 加拿大權利與自由憲章 》 第十五部分禁止基於性傾向的歧視 。", "在地方法院和聯邦上訴法院輸掉了官司以後 ， 艾根和奈斯比特上訴至加拿大最高法院 。"]}\n'

In [70]:
data[0].index(",")

6

In [71]:
pub_test_extract[0]

{'id': 5208,
 'claim': '光學顯微鏡是以電磁學原理來將不可見或難見的微小物放大至肉眼可見的儀器。',
 'predicted_pages_1': ['顯微鏡', '光學顯微鏡'],
 'predicted_evidence': [['顯微鏡', 0],
  ['光學顯微鏡', 0],
  ['顯微鏡', 6],
  ['顯微鏡', 1],
  ['光學顯微鏡', 17]],
 'enough': 0.17492491845041513,
 'predict_label': 0.9793165445327758}

In [76]:
for i in data:
    label_map = ["refute","support","not enough"]
    label_str = i[:i.index("{")]
    evid_str = i[i.index("{"):]
    evid_str = evid_str[:evid_str.index(",")]
    idx = int(evid_str[evid_str.index(" ")+1:])
    label_buf = []
    for l in label_map:
        if l in label_str:
            label_buf.append(l)
    if len(label_buf)>1:
        if pub_test_extract[pub_test_map[idx]]["predicted_evidence"]!=None:
            pub_test_extract[pub_test_map[idx]]["predicted_label"]=label_buf[0]+"s"
        else:
            pub_test_extract[pub_test_map[idx]]["predicted_label"]="NOT ENOUGH INFO"
    else:
        if pub_test_extract[pub_test_map[idx]]["predicted_evidence"]!=None:
            pub_test_extract[pub_test_map[idx]]["predicted_label"]=label_buf[0]+"s"
        else:
            pub_test_extract[pub_test_map[idx]]["predicted_label"]="NOT ENOUGH INFO"
pub_test_extract[:5]

[{'id': 5208,
  'claim': '光學顯微鏡是以電磁學原理來將不可見或難見的微小物放大至肉眼可見的儀器。',
  'predicted_pages_1': ['顯微鏡', '光學顯微鏡'],
  'predicted_evidence': [['顯微鏡', 0],
   ['光學顯微鏡', 0],
   ['顯微鏡', 6],
   ['顯微鏡', 1],
   ['光學顯微鏡', 17]],
  'enough': 0.17492491845041513,
  'predict_label': 0.9793165445327758},
 {'id': 1019,
  'claim': '產絲的蠶或產蜜的蜜蜂爲提供間接經濟利益的昆蟲。',
  'predicted_pages_1': ['昆蟲', '蜂蜜'],
  'predicted_evidence': [['昆蟲', 22],
   ['昆蟲', 23],
   ['蜂蜜', 0],
   ['蜂蜜', 13],
   ['昆蟲', 2]],
  'enough': 0.6171863251365721,
  'predict_label': 0.959430456161499,
  'predicted_label': 'refutes'},
 {'id': 8514,
  'claim': '波蘭西部的綠山城縣平均每平方公里的土地有0人。',
  'predicted_pages_1': ['綠山城縣', '綠山城'],
  'predicted_evidence': [['綠山城縣', 0],
   ['綠山城', 0],
   ['綠山城', 7],
   ['綠山城', 4],
   ['綠山城', 3]],
  'enough': 0.961724066734314,
  'predict_label': 0.959663701057434},
 {'id': 1874,
  'claim': 'Vivien Leigh主演魂斷藍橋中的女配角。',
  'predicted_pages_1': ['魂斷藍橋'],
  'predicted_evidence': [['魂斷藍橋', 0],
   ['魂斷藍橋', 4],
   ['魂斷藍橋', 7],
   ['魂斷藍橋', 6]

In [77]:
pub_test_extract[0]

{'id': 5208,
 'claim': '光學顯微鏡是以電磁學原理來將不可見或難見的微小物放大至肉眼可見的儀器。',
 'predicted_pages_1': ['顯微鏡', '光學顯微鏡'],
 'predicted_evidence': [['顯微鏡', 0],
  ['光學顯微鏡', 0],
  ['顯微鏡', 6],
  ['顯微鏡', 1],
  ['光學顯微鏡', 17]],
 'enough': 0.17492491845041513,
 'predict_label': 0.9793165445327758}

In [78]:
for i in pub_test_extract:
    if 'predicted_label' not in i:
        if i["enough"]>0.05:
            if i['predict_label']>0.5:
                i['predicted_label']="refutes"
            else:
                i['predicted_label']="supports"
        else:
            i['predicted_label']="NOT ENOUGH INFO"

In [84]:
save_keys = ['id','predicted_evidence','predicted_label']
save_lst=[]
for i in pub_test_extract:
    if i['predicted_label']=="not enoughs":
        i['predicted_label']="NOT ENOUGH INFO"
    buf={}
    for key in i:
        if key in save_keys:
            buf[key]=i[key]
    save_lst.append(buf)
save_lst[:2]

[{'id': 5208,
  'predicted_evidence': [['顯微鏡', 0],
   ['光學顯微鏡', 0],
   ['顯微鏡', 6],
   ['顯微鏡', 1],
   ['光學顯微鏡', 17]],
  'predicted_label': 'refutes'},
 {'id': 1019,
  'predicted_evidence': [['昆蟲', 22],
   ['昆蟲', 23],
   ['蜂蜜', 0],
   ['蜂蜜', 13],
   ['昆蟲', 2]],
  'predicted_label': 'refutes'}]

In [85]:
all_pub_test =load_json("../preprocess/public_test.jsonl")
all_pub_test_map = {i["id"]:idx for idx,i in enumerate(all_pub_test)}

In [86]:
save_lst_map = {i["id"]:idx for idx,i in enumerate(save_lst)}
len(save_lst)

921

In [87]:
for idx in all_pub_test_map:
    if idx not in save_lst_map:
        save_lst.append({"id":idx,"predicted_label":"NOT ENOUGH INFO",'predicted_evidence':None})
len(save_lst)

989

In [88]:
with open("../submissions/pub_test_PH100.jsonl","w",encoding="utf8",) as f:
    for i, d in enumerate(save_lst):
        f.write(json.dumps(d, ensure_ascii=False) + "\n")